# Working with geographical data on Google Maps
Source: The Analytics Edge book (MIT)

Dataset: Chicago motor vehicle thefts data from the Chicago Police Dept.

# **Environment Setup**

In [12]:
# datasets to be used: "mvt.csv"
# upload datasets to a folder in Google Drive, e.g., My Drive/Colab Data
# connect to Google Drive, path to datasets is "/content/drive/My Drive/Colab Data/..."
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# install and import required packages
# gmaps tutorials https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
!pip -q install gmaps;
import gmaps
import pandas as pd
## enable support for gmaps widgets (required in the google colab environment)
from google.colab import output
output.enable_custom_widget_manager()

In [14]:
## Credentials
## Here, you need an API code to be able to use the google maps functions
## To obtain the API you will need a google cloud account (90 day free trial)
## Please follow the instructions here:
## https://jupyter-gmaps.readthedocs.io/en/latest/authentication.html
## Once you have your api key, please place it below.
apikey = ''
gmaps.configure(api_key=apikey)

In [15]:
# load chicago maps
chicago_coordinates = (41.88, -87.62)
gmaps.figure(center=chicago_coordinates, zoom_level=12)

Figure(layout=FigureLayout(height='420px'))

In [16]:
# load data as a Dataframe
mvt = pd.read_csv("/content/drive/My Drive/Colab Data/mvt.csv")
mvt.head()

,Date,Latitude,Longitude
0,12/31/12 23:15,41.756284,-87.621645
1,12/31/12 22:00,41.898788,-87.661303
2,12/31/12 22:00,41.969186,-87.767670
3,12/31/12 22:00,41.769329,-87.657726
4,12/31/12 21:30,41.837568,-87.621761


In [17]:
mvt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191641 entries, 0 to 191640
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date       191641 non-null  object 
 1   Latitude   189365 non-null  float64
 2   Longitude  189365 non-null  float64
dtypes: float64(2), object(1)
memory usage: 4.4+ MB


In [18]:
## rearrange mvt to fit gmap format latitude, longitude, first 100 rows of mvt
mvt_heatmap = mvt[['Latitude','Longitude']][:100].dropna()

In [19]:
mvt_heatmap.head()

,Latitude,Longitude
0,41.756284,-87.621645
1,41.898788,-87.661303
2,41.969186,-87.767670
3,41.769329,-87.657726
4,41.837568,-87.621761


In [20]:
# Patch to fix the collections.Iterable issue
import collections
import collections.abc
collections.Iterable = collections.abc.Iterable

## gmap symbol layer
fig = gmaps.figure(center = chicago_coordinates, zoom_level = 10)
heatmap_layer = gmaps.symbol_layer(mvt_heatmap, fill_color='black', stroke_color='black', scale=2)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
## chicago maps heatmap layer
fig = gmaps.figure(center = chicago_coordinates, zoom_level = 10, map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(mvt_heatmap)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
## Create new dataframe with counts for lat lon
latloncounts = round(mvt.dropna(),2).groupby(["Latitude","Longitude"]).count().reset_index().rename(columns = {'Date':"Count"})
latloncounts.head()

,Latitude,Longitude,Count
0,41.64,-87.61,3
1,41.64,-87.54,1
2,41.65,-87.62,54
3,41.65,-87.61,60
4,41.65,-87.60,28


In [23]:
## map with weights
fig = gmaps.figure(center = chicago_coordinates, zoom_level = 10, map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(latloncounts[['Latitude','Longitude']],weights=latloncounts['Count'],
    max_intensity=100, point_radius=4.0)
fig.add_layer(heatmap_layer)
fig.add_traits()
fig

Figure(layout=FigureLayout(height='420px'))